Homework 4 — (15 points)
======
### What to hand in
You are to submit the following things for this homework:
1. A Jupyter notebook containing all code and output (figures and audio). I should be able to evaluate the file to reproduce all output. 
1. Any other data that we tell you to save to a file (e.g. audio files).

### How to hand it in
To submit your lab:
1. Compress all of the files specified into a .zip file. 
1. Name the file in the following manner, firstname_lastname_hw1.zip. For example, Bryan_Pardo_hw1.zip. 
1. Submit this .zip file via Canvas

### Run this code block 1st, to import the needed packages

In [2]:
# This line is a convenience to import most packages you'll need. You may need to import others (eg random and cmath)
import IPython, numpy as np, scipy as sp, matplotlib.pyplot as plt, matplotlib, sklearn, librosa, cmath,math
from IPython.display import Audio
 
# This line makes sure your plots happen IN the webpage you're building, instead of in separate windows.
%matplotlib inline

### Some useful code

####  STFT 

In [3]:
from scipy.fftpack import fft
from scipy.signal import hann

def stft(signal, window_size, hop_size, window_type = 'hann'):
    """
    Computes the short term fourier transform of a 1-D numpy array, where the array 
    is windowed into a set of subarrays, each of length window_size. The distance between
    window centers (in samples) is given by hop_size. The type of window applied is
    determined by window_type. This returns a 2-D numpy array where the ith column
    is the FFT of the ith window. Each column contains an array of complex values.
    
    Input Parameters
    ----------------
    signal: The 1-d (complex or real) numpy array containing the signal
    window_size: an integer scalar specifying the number of samples in a window
    hop_size: an integer specifying the number of samples between the start of adjacent windows
    window_type: a string specifying one of two "hann" or "rectangular"
    
    Returns
    -------
    a 2D numpy array of complex numbers where the array column is the FFT of the ith window,
    and the jth element in the ith column is the jth frequency of analysis.
    """
    
    # figure out how many hops
    length_to_cover_with_hops = len(signal) - window_size;
    assert (length_to_cover_with_hops >= 0), "window_size cannot be longer than the signal to be windowed"
    num_hops = 1 + length_to_cover_with_hops/hop_size;
    
    # make our window function
    if (window_type == 'hann'):
        window = sp.signal.hann(window_size, sym=False)
    else:
        window = np.ones(window_size)
    
    stft = [0]*num_hops
    # fill the array with values 
    for hop in range(num_hops):
        start = hop*hop_size
        end = start + window_size
        unwindowed_sound = signal[start:end]
        windowed_sound =  unwindowed_sound * window
        stft[hop]= fft(windowed_sound, window_size) 
    return np.array(stft).T

#### ISTFT

In [4]:
from scipy.fftpack import ifft

def istft(X, hop_size):
    """
    Takes a 2-D numpy array representing an STFT of some signal, where stft[i] 
    is the FFT of the ith window as input and stft[i,k] is the kth frequency of analysis.
    Performs an inverse FFT on each window and then does overlap & add resynthesis to rebuild 
    the original signal the STFT was built from.
    
    Input Parameters
    ----------------
    X: a 2-D numpy array of complex numbers representing an STFT, where the ith 
    column is the FFT of the ith window, and the jth row is the jth frequency of analysis.
        
    hop_size: an integer specifying the number of samples between the start of adjacent windows.
        
    Returns
    -------
    a 1-d numpy array of (possibly complex) values representing the original signal used to make X
    """
    
    # make an empty signal of the appropriate length
    window_size,num_hops = X.shape
    signal_length = (num_hops-1)*hop_size + window_size 
    signal = np.zeros(signal_length,dtype='complex');
    
    #fill the signal
    for n in range(num_hops):
        start = n * hop_size
        end = start + window_size
        signal[start:end] = signal[start:end] + ifft(X[:,n])
    return signal

#### Spectrogram (plotting function with zooming options)

In [5]:
def plt_spectrogram(X,win_length, hop_size, sample_rate, zoom_x=None, zoom_y=None,tick_labels='time-freq'):
    """
    Plots the log magnitude spectrogram.
    
    Input Parameters:
    ------------------
    X: 2D complex numpy array containing the stft values. Rows correspond to frequency bins and columns to time frames.
    win_length: the length of the analysis window
    hop_size: the hop size between adjacent windows
    sample_rate: sampling frequency
    tick_labels: the type of x and y tick labels, there are two options:
                 'time-freq': shows times (sec) on the x-axis and frequency (Hz) on the y-axis (default)
                 'bin-frame': shows time frame number on the x-axis and frequency bin number on the y-axis
                
    zoom_x: 1 by 2 numpy array containing the range of values on the x-axis, e.g. zoom_t=np.array([x_start,x_end])
    zoom_y: 1 by 2 numpy array containing the range of values on the y-axis, e.g. zoom_f=np.array([y_start,y_end])
    
    
    Returns:
    ---------
    times: 1D real numpy array containing time instances corresponding to stft frames
    freqs: 1D real numpy array containing frequencies of analyasis up to Nyquist rate
    2D plot of the magnitude spectrogram
    """
    
    # Find the size of stft
    Nf,Nt=np.shape(X)
    
    # Compute the log magnitude spectrogram
    X=20*np.log10(np.abs(X))
    
    # Extract the first half of the spectrum for each time frame
    X=X[0:Nf/2]
    Nf=np.shape(X)[0]
        
    # Generate time vector for plotting
    times=(hop_size/float(sample_rate))*np.arange(Nt)
    
    # Generate frequency vector for plotting
    freqs=(float(sample_rate)/win_length)*np.arange(Nf)
    
    # Generate time and frequency matrices for pcolormesh
    times_matrix,freqs_matrix=np.meshgrid(times,freqs)
    
    # Plot the log magnitude spectrogram
    plt.title('Log magnitude spectrogram')
    if tick_labels == 'bin-frame':
        plt.pcolormesh(X)
        plt.xlabel('Time-frame Number')
        plt.ylabel('Frequency-bin Number')
    else:
        plt.pcolormesh(times_matrix,freqs_matrix,X)
        plt.xlabel('Time (sec)')
        plt.ylabel('Frequency (Hz)')

        
    # Zoom in on the plot if specified
    if zoom_x is None and zoom_y is None:
        plt.axis('tight')
        
    if zoom_x is not None:
        plt.xlim(zoom_x)
        
    if zoom_y is not None:
        plt.ylim(zoom_y)
        
    return    

### Time-Frequency Masking

#### 1. (1 point) Load *police_noisy.wav*. The audio contains a ringing noise somewhere. Calculate the STFT of the audio signal and the magnitude spectrogram respectively denoted by *X* and *V*. Keep in mind that the magnitude spectrogram includes only the first half of the spectrum for each time frame. Use the function *plt_spectrogram* provided above to plot the *log magnitude spectrogram* of the audio signal. Identify the region of the ringing noise. It should be a square of about 30 frequency bins by 30 time frames. Write down the frequency and time indices in your report. 

#### * Note: we need to find the coordinates of the noisy region in terms of frequency bin number and time frame number, not in Hz and seconds. You can use the tick_labels parameter to set the values on the x-axis and y-axis to the required units. Moreover, zoom-x and zoom-y parameters can be set to narrower ranges in order to give better estimates of the region boundaries. Be careful about the consistency of zoom-x and zoom-y units with tick-labels.*

your answer goes here

In [ ]:
# your code goes here


#### 2. (1 points) Use the frequency and time indices you found in Question 1 to create a simple binary time-frequency mask to remove the region of the ringing noise. The binary mask, *M*, should be of the same size as the magnitude spectrogram *V*. Derive a full (two-sided) time-frequency mask, *M_full*, from *M* and apply it to the STFT of the noisy audio. Compute the denoised signal by tranforming the masked STFT, *X_masked*, back to time domain. Plot the magnitude spectrogram of the denoised audio and check that the region of the ringing noise has been removed. 

#### Explain why  you should apply the time-frequecy mask to the STFT and not to the magnitude spectrogram to extract the noise. Why do we need to derive a *full* time-frequency mask?

your answer goes here

In [ ]:
# your code goes here

### Beat Spectrum

#### 3. (1 point) We have provided an implementation of the autocorrelation function. Note that we normalize by the number of nonzero additons. Explain the effect this has (compared to the standard autocorrelation in numpy) and tell us why you think we did this.

In [ ]:
def acorr(x):
    """
    Takes a 1D numpy array and returns the autocorrelation. 
    
    Input Parameter:
    ----------------
    x: 1D numpy array of length Lx
    
    Ouput Parameter:
    ----------------
    x_acorr: 1D numpy array of length x_len containing the values of the autocorrelation function of x
    
    Note: the actual length of autocorrelation function is (2*x_len)-1, but since this function is symmetric
          we can cut off (x_len)-1 samples and return one side of length x_len without losing information. 
    """
    
    x_len=np.size(x)
    x_pad=np.concatenate([x,np.zeros(x_len-1)])
    x_acorr=ifft(np.abs(fft(x_pad))**2).real
    x_acorr=x_acorr[0:x_len]
    
    x_acorr=x_acorr/(np.arange(x_len)[::-1]+1)  # normalize by the number of nonzero additions
            
    return x_acorr

# Test case: 
# Note: the numpy correlate function is not normalized so the results seem different. If you comment out the 
#       normalization line above the outputs will be the same.
test_array=np.arange(10)
print 'Output of the accor function:  '+ str(acorr(test_array))
print 'Output of the correlate function:  '+str(np.correlate(test_array,test_array,"full")[len(test_array)-1:2*len(test_array)])


#### 4. (2 points) Implement the beat spectrum. Use the autocorrelation function from Question 3. Do not square the magnitude spectrogram within your function; it will be squared outside the function, in the *repet* function. Apply the function to the spectrogram of the noisy as well as the denoised signals. Plot both beat spectra. Do you observe any noticeable difference between the plots? What information does the beat spectrum provide regarding the ringing noise?

your answer goes here

In [ ]:
def beat_spectrum(S):
    """
    Computes the beat spectrum of a music mixture by applyting the autocorrelation to all frequency channels (rows)
    of the spectrogram and taking the average. Note that the assumption is that the mixture is composed of a repeating
    background and a non-repeating foreground. Therefore, the spikes in the beat spectrum represent the repetitive 
    events in the spectrogram.  This function must PLOT the beat spectrum for viewing, since you'll be looking at the
    resulting plot to pick the repeating period used by REPET. 
    
    Input Parameter:
    ----------------
    S: 2D numpy array containng the spectrogram of a signal
    
    Output Parameter:
    ------------------
    beat_spec: 1D numpy array containing the beat spectrum 
    
    """
    # your code goes here
    

#### 5. (1 point) Explain how you would find the period of the repeating structure from the beat spectrum. Remember that the repeating period does not necessarily correspond to the first highest peak. Remember also that the beat spectrum goes from lag 0 to lag n-1, where n is the length of the input signal.

your answer goes here

### REPET Algorithm

#### 6. (2 point) Implement a repeating-segment modeling function (Hint: use the numpy *median* function). Note that after segmentation of the spectrogram, the last segment is generally shorter (in time frames) than the other segments, because the number of time frames is not necessarily an integer multiple of the repeating period; Think about how you would handle that.

In [5]:
def repeating_segment(V,p):
    """
    Computes the repeating-segment model using the period obtained from the beat spectrum. The repeating segment
    is that median background we'll be using to compare to the spectrum. Its time duration will be the length of
    one period that you learned from the beat spectrum. 
    
    Input Parameters:
    -----------------
    V: 2D numpy array containing the magnitude spectrogram of the mixture
    p: scalar, repeating period (in number of samples) found from the beat spectrum 
    
    Output Parameter:
    S: 2D numpy array containing the repeating segment 
    """  
    # your code goes here

#### 7. (2 point) Implement a repeating spectrogram modeling function (Hint: use the min function). Again, the last segment in your spectrogram is generally shorter than the repeating segment model. 

In [4]:
def repeating_spectrogram(V,S):
    """
    Builds the repeating-spectrogram model using the repeating segment model computed in the previous question.
    This is composed of multiple copies of the repeating segment, tiled together in the time-dimension so that it
    is the exact same size as the input signal's spectrogram.
    
    Input Parameters:
    -----------------
    V: 2D numpy array containing the magnitude spectrogram of the mixture
    S: 2D numpy array containing the repeating segment model 
    
    Output Parameter:
    ------------------
    W: 2D numpy array containing the repeating-spectrogram 
    """
    # your code goes here

#### 8. (1 point) Implement a repeating-mask modeling function (Hint: use a division). Note that there could be zero values in your spectrogram. Think about how to handle that. 

In [3]:
def repeating_mask(V,W):
    """
    Before we do our separation, we need to compare our repeating_spectrogram to the original spectrogram of the signal
    to determine how to assign the energy in each time-frequency point (each element in your matrix). This function
    does that and makes a mask that will be applied to the spectrum. Note that unlike the denoising case in the first 
    question, here we are computing a "soft mask" with values ranging between 0 and 1.
    
    Input Parameters:
    -----------------
    V: numpy 2D array containing the magnitude spectrogram 
    W: numpy 2D array containing the repeating spectrogram
    
    Output Parameter:
    M: numpy 2D array containing the repeating mask 
    """
    # your code goes here

### Music-voice Separation

#### 9.  (2 points) The function provided below runs the REPET algorithm using functions you wrote in previous questions. REPET builds a soft time-frequency mask (with values between 0 and 1), but we can also derive a binary time-frequency mask from it (with values of only 0s and 1s). To do so, we simply use a threshold value between 0 and 1 in the soft mask, so that values above the threshold will be forced to 1, while values below the threshold will be forced to 0. A binary mask can help to improve the perception of separation, but it will also increase the separation artifacts. 

#### Select the value of the repeating period in samples and the value of the masking threshold. Try different values of the masking threshold and compute the background and foreground audio estimates. Listen to the estimate and plot the magnitude spectrograms of the background and foreground estimates for each masking threshold you try. Write down the value of the masking threshold that gave you the best separation results.

#### Describe the separation results that you have obtained. Are the repeating musical background and the non-repeating vocal foreground well separated? Did you get any kind of audio artifacts? What did happen to the ringing noise? Why? 

In [ ]:
def repet(file_path, rep_period, mask_thr):
    """
    Runs the REPET algorithm using functions implemented in qustions 3 through 8. NOTE: In the real REPET algorithm
    it also determines the repeating period from the beat spectrum by building a peak picker to find the best
    period from the beat spectrum. This seemed like too much for a homework, so we're making YOU the peak picker. 
    You'll have to run the beat_spectrum function before calling repet and input the value for rep_period.
    
    Input Parameters:
    ------------------
    file_path: string, path to the audio file to separate
    rep_period: scalar, repeating period in number of samples
    mask_thr: scalar, a value between 0 and 1 for thresholding the soft mask
    
    Output:
    --------
    Separated background and foreground magnitude spectrogram plots 
    
    Returns: 
    ---------
    Separated background and foreground time signals
    """
    
    # Load the noisy mixture
    signal, sample_rate = librosa.load(file_path, 44100) 

    # Comput the stft of the audio signal 
    win_length_sec=0.04 # 40 msec window
    win_length_samp=int(2**np.ceil(np.log2(win_length_sec*sample_rate))) # next power2 of winow length in no. of samples
    n_fft=win_length_samp
    hop_size=win_length_samp/2 # 50% overlap
    win_type=sp.signal.hanning
    X = stft(signal, win_length_samp, hop_size, window_type = 'hann')

    # Compute the magnitude spectrogram (half spectrum)
    Vm = np.abs(X[0:win_length_samp/2+1,:])
    Nf,Nt=Vm.shape
    
    
    # Compute and plot the beat spectrum
    beat_spec = beat_spectrum(Vm**2)
    beat_spec = beat_spec/beat_spec[0] # normalization
    
    plt.figure()
    plt.plot(beat_spec)
    plt.grid('on')
    plt.axis('tight')
    plt.xlabel('Lag (sample number)')
    plt.title('Beat spectrum of the noisy mixture')
    
    ### REPET Algorithm ###
    
    # Compute the repeating soft mask
    Sm=repeating_segment(Vm,rep_period)
    Wm=repeating_spectrogram(Vm,Sm)
    soft_mask=repeating_mask(Vm,Wm)
    
    #plt.figure()
    #plt.pcolormesh(soft_mask)
    
    # Compute the repeating binary mask
    binary_mask=np.zeros(soft_mask.shape)
    binary_mask[soft_mask>=mask_thr]=1
    
    
    # Estimate the background and foreground via masking
    binary_mask=np.vstack([binary_mask,np.flipud(binary_mask[1:-1,:])])
    
    X_bg = binary_mask*X+1e-16  # background STFT
    signal_bg = np.real(istft(X_bg, hop_size)) # background time signal
    
    X_fg = (1-binary_mask)*X+1e-16  # foreground STFT
    signal_fg = np.real(istft(X_fg, hop_size)) # foreground time signal
            
    # Plot the log magnitude spectrograms and the binary mask
    plt.figure()    
    plt.subplot(211)
    plt_spectrogram(X,win_length_samp, hop_size, sample_rate,tick_labels='time-freq')
    plt.title('Mixture')
        
    plt.subplot(212)
    plt.pcolormesh(binary_mask[0:Nf,:])
    plt.xlabel('Time (sec)')
    plt.ylabel('Frequency (Hz)')
    plt.title('Binary mask')
    plt.axis('tight')
    
    plt.figure()
    plt.subplot(211)
    plt_spectrogram(X_bg,win_length_samp, hop_size, sample_rate,tick_labels='time-freq')
    plt.title('Background')
    
    plt.subplot(212)
    plt_spectrogram(X_fg,win_length_samp, hop_size, sample_rate,tick_labels='time-freq')
    plt.title('Foreground')
    
    return signal_bg,signal_fg

    

In [ ]:
# your code goes here


your answers go here.

#### 10. (2 points) Go out on the web and find an example of music where REPET works well to separate out the voice and an example where it works poorly and do the following:
1. Crop each example to be between 10 and 30 seconds (pick a length appropriate for making your point)
1. Save each cropped example at the sample rate 22050 Hz
1. Make each example monophonic (ie one channel, not two)
1. Include both examples with your submission
1. Write the code to call repet, separate the sources and play them.

your answer goes here

In [ ]:
# your code goes here